In [1]:
import ee
import geemap
import math
ee.Initialize()

In [79]:
javascript_code = """
var test = loss1.eq(1)

var test2 = loss2.neq(1)

var comb = test.and(test2)
Map.addLayer(comb, {min: 0, max: 1, palette: ['white', 'blue']}, 'test')
"""

In [80]:
geemap.js_snippet_to_py(javascript_code)

In [ ]:
ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
activeMines = ee.FeatureCollection("users/raymondeah/cod_mines_curated_all_opendata_p_ipis")
roi1 = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

roi3 = ee.Geometry.Polygon(
        [[[30.246670960050185, 1.7911944738716732],
          [30.246670960050185, 1.7103797163160706],
          [30.356362579923232, 1.7103797163160706],
          [30.356362579923232, 1.7911944738716732]]])

modis = ee.ImageCollection("MODIS/006/MOD13Q1")

ndviVis = {
  'min': 0.0,
  'max': 8000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}

rgbVis = {
  'min': 8097.96,
  'max': 12128.04,
  'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}

NameError: name 'ee' is not defined

In [3]:
"""
Segment the given geometry into squares of given size (in km?)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    
    r_earth = 6378
    dy, dx = size, size
    pi = math.pi
    
    coords = geometry.coordinates().getInfo()[0][:-1]
    # coordinate structure: 0-top left, 1-top right, 2, top 
    top = coords[2][1] # high lat
    left = coords[0][0] # right lon
    
    width = ee.Geometry.Point(coords[0]).distance(ee.Geometry.Point(coords[1])).divide(1000 * size).getInfo()
    height = ee.Geometry.Point(coords[1]).distance(ee.Geometry.Point(coords[2])).divide(1000 * size).getInfo()
    
    #print(width, height)
    
    for i in range(int(height) + 1):
        left = coords[0][0]
        for j in range(int(width) + 1):
            new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            new_lat = left  + (dy / r_earth) * (180 / pi)
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]])
            
            #filtered = ls8 \
             # .filter(ee.Filter.bounds(geometry)) \
             # .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
            #  .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
             # .select('SR_B.*')

           # composite = filtered.median().clip(square)
            
            segments.append(square)
           #segments.append(composite)
            
            left = new_lat
        top = new_lon
            
    return segments

In [4]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

In [5]:
Map = geemap.Map()
Map.centerObject(roi1, 12)
filtered = ls8 \
    .filter(ee.Filter.bounds(roi1)) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
    .select('SR_B.*')

composite = filtered.median().clip(roi1)
Map.addLayer(composite, rgbVis, '2020 Median Composite')
segments = create_segments(roi1, 1)
print(ee.Feature(segments[1]).geometry().area(1).divide(1e6).getInfo(), 'sq km')
Map.addLayer(ee.FeatureCollection(segments), {'color': 'red'}, 'Squares (server side)')
Map

0.9963367405740445 sq km


Map(center=[3.1257435633122834, 29.589663178011996], controls=(WidgetControl(options=['position', 'transparent…

In [10]:
# big = ee.Geometry.Polygon(
#         [[[29.324961334750263, 3.3062182501952346],
#           [29.324961334750263, 2.9023783494297306],
#           [29.879770905062763, 2.9023783494297306],
#           [29.879770905062763, 3.3062182501952346]]], None, False)

# big2 = ee.Geometry.Polygon(
#         [[[23.93083329745621, 5.799063854338723],
#           [23.93083329745621, -0.38729587002473354],
#           [31.48942704745621, -0.38729587002473354],
#           [31.48942704745621, 5.799063854338723]]])

# Map = geemap.Map()
# Map.centerObject(big2, 7)

# segments = create_segments(big2, 10)
# #print('area:', segments[1].area(1).divide(1e6).getInfo(), 'km^2')
# segs = ee.FeatureCollection(segments)
# Map.addLayer(segs, {'color': 'red'}, 'segments')
# Map

In [7]:
segs = create_segments(roi3, 1)

def landsat(feature):
    filtered = ls8 \
      .filter(ee.Filter.bounds(feature.geometry())) \
      .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
      .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
      .select('SR_B.*') \
      .median() \
      .clip(feature.geometry())

    return filtered

def ndvi_before(feature):
    ndvi = modis \
        .filter(ee.Filter.bounds(feature.geometry()))\
        .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
        .select('NDVI') \
        .median() \
        .clip(feature.geometry())
    
    return ndvi

def ndvi_after(feature):
    ndvi = modis \
        .filter(ee.Filter.bounds(feature.geometry()))\
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .select('NDVI') \
        .median() \
        .clip(feature.geometry())
    
    return ndvi

composites = ee.ImageCollection(ee.FeatureCollection(segs).map(landsat))
ndvi2000 = ee.ImageCollection(ee.FeatureCollection(segs).map(ndvi_before))
ndvi2020 = ee.ImageCollection(ee.FeatureCollection(segs).map(ndvi_after))

Map = geemap.Map()
Map.centerObject(roi3, 10)

Map.addLayer(composites, rgbVis, 'composites')
Map.addLayer(ndvi2000, ndviVis, 'NDVI 2000')
Map.addLayer(ndvi2020, ndviVis, 'NDVI 2020')
Map.addLayer(activeMines, {'color': 'blue'}, 'Active Mines')
Map.addLayer(ee.FeatureCollection(segs), {'color': 'red'}, 'squares')
Map

Map(center=[1.7507876063752705, 30.30151676998732], controls=(WidgetControl(options=['position', 'transparent_…

In [8]:
# before = ndvi2000.first().select('NDVI').lt(5000)
# after = ndvi2020.first().select('NDVI').lt(5000)
# change = before.neq(1).And(after.eq(1))

# areaImage = change.multiply(ee.Image.pixelArea())
    
# area = areaImage.reduceRegion(**{
#     'reducer': ee.Reducer.sum(),
#     'geometry': change.geometry(),
#     'scale': 10,
#     'maxPixels': 1e10
# })
    
# area = ee.Number(area.get('NDVI')).divide(1e6)
# totalArea = ndvi2020.first().geometry().area(1).divide(1e6)
# percent = area.divide(totalArea).multiply(100)
# print(percent.getInfo())

# after.set('percent loss', percent)
# x = after.get('percent loss')
# print(x)

# Map.addLayer(change, {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'changed')
# Map

In [13]:
# temp = []
# for img1, img2, ls in zip(m2000, m2020, c):
#     before = img1.select('NDVI').lt(5000)
#     after = img2.select('NDVI').lt(5000)
#     change = before.neq(1).And(after.eq(1))
    
#     areaImage = change.multiply(ee.Image.pixelArea())
    
#     area = areaImage.reduceRegion(**{
#         'reducer': ee.Reducer.sum(),
#         'geometry': change.geometry(),
#         'scale': 10,
#         'maxPixels': 1e10
#     })
    
#     area = ee.Number(area.get('NDVI')).divide(1e6)
#     totalArea = img1.geometry().area(1).divide(1e6)
#     percent = area.divide(totalArea).multiply(100)
#    # print(prcent.getInfo())
#     imgx = ls.set('percent', percent)
#     temp.append(imgx)
      
def select_threshold(img):
    return img.select('NDVI').lt(5000)

before = ndvi2000.map(select_threshold)
after = ndvi2020.map(select_threshold)

filter2 = ee.Filter.equals({
  'leftField': 'NDVI',
  'rightField': 'NDVI'
})

simpleJoin = ee.Join.inner()
innerJoin = ee.ImageCollection(simpleJoin.apply(before, after, filter2))

#change = ee.ImageCollection.map(change(before, after))

Map.addLayer(before, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'before')
Map.addLayer(after, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'after')

co = ee.ImageCollection(temp)
d = co.filter(ee.Filter.gt('percent', 5))
Map.addLayer(d, rgbVis, 'filtered squares')
Map

Map(bottom=519508.0, center=[1.7438103112712384, 30.297595879763534], controls=(WidgetControl(options=['positi…

In [15]:
all = ee.Geometry.Polygon(
        [[[22.92587933089792, 5.5911393992628495],
          [22.92587933089792, -13.776973382582892],
          [31.45126995589792, -13.776973382582892],
          [31.45126995589792, 5.5911393992628495]]])

squares = create_segments(all, 100)
focus = squares[27]
focus2 = create_segments(focus, 10)

def test_map(feature):
    ndvi = modis \
        .filter(ee.Filter.bounds(feature.geometry()))\
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .select('NDVI') \
        .median() \
        .clip(feature.geometry())
    
    return ndvi

Map = geemap.Map()
Map.centerObject(all, 5)
Map.addLayer(all, {'color': 'red'}, 'all')
Map.addLayer(ee.FeatureCollection(squares), {'color': 'red'}, '100x100')
#Map.addLayer(big, {'color': 'blue'}, 'ref')
#Map.addLayer(activeMines, {'color': 'green'}, 'Active Mines')
#Map.addLayer(focus, {'color': 'purple'}, 'focus')
#Map.addLayer(ee.FeatureCollection(focus2), {'color': 'pink'}, 'focus squares')

#test = ee.ImageCollection(ee.FeatureCollection(focus2).map(test_map))
#Map.addLayer(test, ndviVis, 'ssss')

smaller = []
for square in squares:
    new = create_segments(square, 10)
    smaller.append(ee.FeatureCollection(new))
    
smaller1 = smaller[:50]
smaller2 = smaller[50:100]
smaller3 = smaller[100:150]
smaller4 = smaller[150:200]
smaller5 = smaller[200:]

#test = ee.ImageCollection(ee.FeatureCollection(smaller1).flatten().map(test_map))
#Map.addLayer(test, ndviVis, 'NDVI 1')

Map.addLayer(ee.FeatureCollection(smaller1).flatten(), {'color': 'green'}, '10x10')
Map.addLayer(activeMines, {'color': 'blue'}, 'Active Mins')
# Map.addLayer(ee.FeatureCollection(smaller2).flatten()), {'color': 'red'}, 'smaller squares 2')
# Map.addLayer(ee.FeatureCollection(smaller3).flatten()), {'color': 'red'}, 'smaller squares 3')
# Map.addLayer(ee.FeatureCollection(smaller4).flatten()), {'color': 'red'}, 'smaller squares 4')
# Map.addLayer(ee.FeatureCollection(smaller5).flatten()), {'color': 'red'}, 'smaller squares 5')
Map

Map(center=[-4.064367300737955, 27.18857464339792], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# def percent_change(l):
#     img1 = l.get(0)
#     img2 = l.get(1)
    
#     before = img1.select('NDVI').lt(5000)
#     after = img2.select('NDVI').lt(5000)
    
#     change = before.eq(1).And(after.neq(1))
    
#     areaImage = change.multiply(ee.Image.pixelArea())
    
#     area = areaImage.reduceRegion(**{
#         'reducer': ee.Reducer.sum(),
#         'geometry': change.geometry(),
#         'scale': 10,
#         'maxPixels': 1e10
#     })
    
#     area = ee.Number(area.get('NDVI')).divide(1e6)
#     totalArea = ee.Number(img.geometry().area(1).divide(1e6))
    
#     percent = area.divide(totalArea).multiply(100)
#     return img1, img2.set('percent loss', percent)
    

In [10]:
#ndvi2020 = ndvi2020.map(percent_change(ndvi2000, ndvi2020))

#filtered = ndvi2020.filter(ee.Filter.gt('percent loss', 0))

#Map.addLayer(filtered, ndviVis, 'P')
#Map
#l = [ndvi2000, ndvi2020]

#c = ee.List(l)
#print(c.get(0))

#test = c.map(percent_change)

#print(test)